In [47]:
import os
import matplotlib.pyplot as plt
from skimage import io
import cv2

In [48]:
def apex_from_mask(mask):
    height, width = mask.shape

    # Calculate the middle row index
    middle_row = height // 2

   # Initialize variables to track the indices of -1 values
    first_neg_one_idx = None
    last_neg_one_idx = None

    # Iterate over the rows from the middle row to the bottom row
    for y in range(middle_row, height):
        row = mask[y, :]
        # Check if there are any -1 values in the row
        if 1 in row:
            # If this is the first row with -1 values, record the indices and the line index
            if first_neg_one_idx is None:
                first_neg_one_idx = np.where(row == 1)[0][0]
                last_neg_one_idx = np.where(row == 1)[0][-1]
                line_idx = y
                center_idx = (first_neg_one_idx + last_neg_one_idx) // 2
                return (line_idx, center_idx )
      
    for y in range(0, middle_row):
        row = mask[y, :]
        # Check if there are any -1 values in the row
        if 1 in row:
            # If this is the first row with -1 values, record the indices and the line index
            if first_neg_one_idx is None:
                first_neg_one_idx = np.where(row == 1)[0][0]
                last_neg_one_idx = np.where(row == 1)[0][-1]
                line_idx = y
                center_idx = (first_neg_one_idx + last_neg_one_idx) // 2
                return (line_idx, center_idx )

    return None

def compute_mask(mask):
    res = mask.copy()
    res = np.where(res >= 0.95, 1, res)
    res = np.where(res < 0.95, 0, res)
    return res

In [49]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import numpy as np

class MeanMetricWrapper(tf.keras.metrics.Mean):
    def __init__(self, name='mean_metric', dtype=None):
        super(MeanMetricWrapper, self).__init__(name=name, dtype=dtype)

    def get_config(self):
        return {'name': self.name}

    @classmethod
    def from_config(cls, config):
        return cls(name=config['name'])

model = tf.keras.models.load_model("first_full_model", custom_objects={'MeanMetricWrapper': MeanMetricWrapper})

In [53]:
import cv2

# Ouvrir la vidéo
video_capture = cv2.VideoCapture("bruit.avi")

# Obtenir les dimensions de la vidéo
frame_width = int(video_capture.get(3))
frame_height = int(video_capture.get(4))

# Créer un objet VideoWriter pour enregistrer la nouvelle vidéo modifiée au format AVI
output_video = cv2.VideoWriter("bruit_modifie.avi",
                               cv2.VideoWriter_fourcc('M','J','P','G'),
                               30,  # Nombre d'images par seconde de la nouvelle vidéo
                               (frame_width, frame_height))


# Parcourir toutes les images de la vidéo
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break
        
    frame_corrected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Correction de balance des blancs

    img_dem = (cv2.resize(frame_corrected*1./255, (224,224)))
    img_dem = np.expand_dims(img_dem, axis=0)

    res_dem = model.predict(img_dem)[0]
    orange_lines = compute_mask(res_dem[:,:,2])
    white_lines = compute_mask(res_dem[:,:,1])


    img_res = img_dem[0].copy()
    img_res[orange_lines != 0] = [1, 0, 0]
    img_res[white_lines != 0] = [0, 1, 0]

    try:
        cv2.circle(img_res, apex_from_mask(orange_lines)[::-1], 3, (0, 0, 1), 3)
    except:
        try:
            cv2.circle(img_res, apex_from_mask(white_lines)[::-1], 3, (0, 0, 1), 3)
        except:
            pass
        
    toto = cv2.resize((img_res*255).astype('uint8'), (1280,720))
    output_video.write(toto)

        
# Libérer les ressources
video_capture.release()
output_video.release()

1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 55ms/step


1/1 [==============================] - 0s 57ms/step


1/1 [==============================] - 0s 57ms/step


1/1 [==============================] - 0s 56ms/step


...

1/1 [==============================] - 0s 70ms/step


1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 59ms/step


[mpeg4 @ 0x55ba4145c000] ac-tex damaged at 61 42
[mpeg4 @ 0x55ba4145c000] Error at MB: 3463


1/1 [==============================] - 0s 60ms/step
